# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [89]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from sklearn.cluster import KMeans
%matplotlib inline

#Print working directory
os.getcwd()

'/Users/lanapalmer/Documents/Jupyter/Springboard/Guided Capstone'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [90]:
#Load csv
df = pd.read_csv('step3_output.csv')

#Visual Check
df.head(5)

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [91]:
#Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               330 non-null    object 
 1   state              330 non-null    object 
 2   summit_elev        330 non-null    int64  
 3   vertical_drop      330 non-null    int64  
 4   base_elev          330 non-null    int64  
 5   trams              330 non-null    int64  
 6   fastEight          330 non-null    float64
 7   fastSixes          330 non-null    int64  
 8   fastQuads          330 non-null    int64  
 9   quad               330 non-null    int64  
 10  triple             330 non-null    int64  
 11  double             330 non-null    int64  
 12  surface            330 non-null    int64  
 13  total_chairs       330 non-null    int64  
 14  Runs               330 non-null    float64
 15  TerrainParks       330 non-null    float64
 16  LongestRun_mi      330 non

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [92]:
dfo = df[['state']] # select state column
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
df.head()

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,Alyeska Resort,3939,2500,250,1,0.0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,9200,0,0.0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [93]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend', 'AdultWeekday'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df[['AdultWeekend']]

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled = scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [94]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = np.ravel(y)

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [95]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score, mean_absolute_error

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [96]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test

y_pred = lm.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [97]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

explained_variance_score(y_test, y_pred)

0.3039001700295787

In [98]:
mean_absolute_error(y_test, y_pred)

14.339295659629709

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [99]:
print(lm.intercept_)

64.30443219731384


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [100]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coeffs = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coeffs.sort_values(by=['Coefficient'], ascending=False)

,Coefficient
total_chairs,3.706898e+12
fastQuads,1.405293e+12
surface,1.316655e+12
double,1.160285e+12
triple,1.035054e+12
quad,8.388728e+11
state_New York,4.994293e+11
state_Michigan,4.713256e+11
fastSixes,4.166000e+11
state_Colorado,4.152651e+11


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [101]:
#Load csv
df2 = pd.read_csv('step3_output.csv')

#Visual Check
df2.head(5)

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [102]:
df2.drop(['state'], axis=1, inplace=True)

#Visual Check
df2.head(5)

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,3939,2500,250,1,0.0,0,2,2,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,11500,2300,9200,0,0.0,1,0,2,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [103]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X2 = df2.drop(['Name','AdultWeekend', 'AdultWeekday'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y2 = df2[['AdultWeekend']]

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X2)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled2 = scaler.transform(X2) 

In [104]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y2 = np.ravel(y2)

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_scaled2, y2, test_size=0.25, random_state=1)

In [105]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train2,y_train2)

In [106]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred2 = lm.predict(X_test2)

#Visual Check
print(y_pred2)

[ 57.0839085   77.27898436  86.74580872  85.63821409  84.79455523
  73.18542655  63.04093667  56.84607489  69.72793287  77.24286029
  52.06083062  80.77789208  61.96104053  59.89391162  56.99377868
  84.17090219  52.27675317  72.41992704  61.36468933  50.80147757
  83.10606819  46.79329117  51.37287944  64.80533277  59.72247324
  92.44239992  69.45544697 170.63515359  55.00294638  63.52445175
  54.50759587  88.48884638  72.54658514  63.68367202  70.37664175
  54.64143886  42.44874086  51.05267414  50.38123164  43.88927049
  53.21268796  66.14600377  69.59708318  76.9568124   64.14262739
  60.77037757  79.13908079 105.13614885  49.63779209  67.33942094
  49.36943212  78.70451264  65.2383585   43.50575202  50.91914276
  47.24862581  80.10575038  52.33855175  87.34944798  84.29922799
  59.87606006  63.18661737  51.10477376  56.52301056  67.57722103
  53.95123619  51.29501003  69.36183064  57.330903    60.48390742
  61.97923652 101.98721839  49.7787993   54.98129621  57.82387963
  56.00234

In [107]:
# You can plug y_test and y_pred into the functions to evaluate the model

explained_variance_score(y_test2, y_pred2)

0.31781276283239823

In [108]:
mean_absolute_error(y_test2, y_pred2)

14.84955079028135

In [109]:
print(lm.intercept_)

64.31912206810779


In [110]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coeffs = pd.DataFrame(abs(lm.coef_), X2.columns, columns=['Coefficient'])
coeffs.sort_values(by=['Coefficient'], ascending=False)

,Coefficient
vertical_drop,8.705734
clusters,6.622147
Snow Making_ac,6.264761
base_elev,5.537120
projectedDaysOpen,3.594007
quad,3.576679
triple,3.252733
fastQuads,2.784778
surface,2.757751
trams,2.497946


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [111]:
#Load csv
df3 = pd.read_csv('step3_output.csv')

#Visual Check
df3.head(5)

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [112]:
df3.drop(['state', 'summit_elev', 'base_elev'], axis=1, inplace=True)

#Visual Check
df3.head(5)

,Name,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,2500,1,0.0,0,2,2,0,0,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,Eaglecrest Ski Area,1540,0,0.0,0,0,0,0,4,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,Hilltop Ski Area,294,0,0.0,0,0,0,1,0,2,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,Arizona Snowbowl,2300,0,0.0,1,0,2,2,1,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,Sunrise Park Resort,1800,0,0.0,0,1,2,3,1,0,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [113]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekday' from the df
X3 = df3.drop(['Name','AdultWeekend', 'AdultWeekday'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y3 = df3[['AdultWeekday']]

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X3)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled3 = scaler.transform(X3) 

In [114]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y3 = np.ravel(y)

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [115]:
#Fit model
lm = linear_model.LinearRegression()
model = lm.fit(X_train3,y_train3)

In [116]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred3 = lm.predict(X_test3)

#Visual Check
print(y_pred3)

[ 55.69041853 106.69359235  72.04369001  77.51390485  87.25120954
  63.61656599  62.37810163  53.95323591  68.58055524  75.66453962
  49.45207624  92.35106306  59.89085798  67.65819196  55.71922712
  91.25413923  47.76549177  65.83885603  60.14647321  47.30369978
 104.23564558  45.6671031   53.20604353  65.10118513  55.5075572
  92.32542829  80.37108259 171.16478376  56.53392438  70.40086774
  45.630482   103.80833845  74.10326032  63.45384626  71.3928111
  44.50975446  39.97288923  49.22386579  51.0009654   39.36229353
  49.85033063  59.31200056  64.86766462  81.75194196  67.80553083
  77.85826521  85.00401716 107.43675642  50.80138044  58.24950056
  49.61278181  71.33079938  68.80638532  50.48973493  50.02611188
  42.24217634  80.37767438  53.11095075  83.05281477 106.31785993
  57.1013072   56.57225446  58.58763532  46.51195173  64.20213728
  41.38560896  46.47594099  64.5803111   64.35033063  51.36424665
  67.99242048  95.82201032  57.87083845  56.50975446  61.78563337
  55.8251841

In [117]:
#Evaluate model
explained_variance_score(y_test3, y_pred3)

0.3039001700295787

In [118]:
mean_absolute_error(y_test3, y_pred3)

14.339295659629709

In [119]:
#Print intercept
print(lm.intercept_)

64.30443219731384


In [120]:
#Calculate coefficients
coeffs = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coeffs.sort_values(by=['Coefficient'], ascending=False)

,Coefficient
total_chairs,3.706898e+12
fastQuads,1.405293e+12
surface,1.316655e+12
double,1.160285e+12
triple,1.035054e+12
quad,8.388728e+11
state_New York,4.994293e+11
state_Michigan,4.713256e+11
fastSixes,4.166000e+11
state_Colorado,4.152651e+11


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.348 |14.02 |-|
| Model 2. | 0.305|15.12 |'state'|
| Model 3. |0.265| 15.59|'state','summit_elev','base_elev'|

Model Selection: Model 3